# Bib


@article{burdine1953,
  title={Relative permeability calculations from pore size distribution data: Petroleum Transactions AIME, v. 198},
  author={Burdine, NT},
  year={1953}
}

@article{campbell1974,
  title={A simple method for determining unsaturated conductivity from moisture retention data},
  author={Campbell, Gaylon S},
  journal={Soil science},
  volume={117},
  number={6},
  pages={311--314},
  year={1974},
  publisher={LWW}
}

@article{jensen1998,
  title={Soil water matric potential rather than water content determines drought responses in field-grown lupin (Lupinus angustifolius)},
  author={Jensen, CR and Mogensen, VO and Poulsen, H-H and Henson, IE and Aagot, S and Hansen, E and Ali, M and Wollenweber, B},
  journal={Functional Plant Biology},
  volume={25},
  number={3},
  pages={353--363},
  year={1998},
  publisher={CSIRO Publishing}
}

@article{moldrup2000,
  title={Predicting the gas diffusion coefficient in undisturbed soil from soil water characteristics},
  author={Moldrup, P and Olesen, Torben and Schj{\o}nning, Per and Yamaguchi, T and Rolston, DE},
  journal={Soil Science Society of America Journal},
  volume={64},
  number={1},
  pages={94--100},
  year={2000},
  publisher={Wiley Online Library}
}

@article{rawls1982,
  title={Estimation of soil water properties},
  author={Rawls, Walter J and Brakensiek, Donald L and Saxtonn, KE},
  journal={Transactions of the ASAE},
  volume={25},
  number={5},
  pages={1316--1320},
  year={1982},
  publisher={American Society of Agricultural and Biological Engineers}
}

### Determination of matric potential from particle size distribution and volumetric water content ###

First we will transform SWC to matric potential ($\Psi$).  $\Psi$ Is a better predictor of water availability than volumetric content because it reflects how difficult the water is to remove from the soil \cite{jensen1998} __(more citations for this statement)__. It is also needed to estimate the soil gas duffusion coeeficient which is needed to calculate gas transport using the gradient method.  After estimating $\Psi$ we will estimate the gas diffusion coefficient based on air filled porosities using equations from Molderup et al. \cite{moldrup2000}.

First we will determine the matric potential based on SWC and soil properties.

### Soil particle dsitribution ###
The soil pits are often very close to the base of the tower, rather than scattered about the tower footprint.  Furthermore, since soil composition veries so much over space it hardly seems reasonable to try to estimate the actual values for sand and clay at each sensor location.  Instead we will just define a joint probablitity distribution of sand, clay and silt percentages based on the observed data.  We can draw from it later as a model parameter.

The soil particle sample distributions were determined as a multivariate normal distribution (`tfp.distributions.MultivariateNormalTriL`) parameterized with the means and covariances of the clay, sand, and silt fractions of soil samples for each __site (?)  (also might break it down by horizon if possible)__.  

__These equations are in model1.ipynb, and somewhere else as a stand alone function__
\$\Psi$ was estimated from volumetric soil water content and soil particle distribution using the methods described by Saxton et al. \cite{saxton1976}.  Three equations were used to describe the relationship between matric potential, soil particle distribution across different ranges of volumetric water content.  For $\Psi$ ranging from saturation to air entry ($\Psi_{e}$) the saturation value was used was used.

\begin{equation} \label{saturation}
\theta = \theta_{s} 
\end{equation}

For values from air entry to 10kP linear equations previously described by Rawls et al. \cite{rawls1982} were used with a fixed organic matter content of 0.66% __(do i have organic matter content? if so plug it on in eh?)__.


\begin{equation} \label{rawlA}
A = 100 \; exp(-4.396 - 0.0715 \; C -4.880 \times 10^{-4} S^{2} - 4.285 \times 10^{-5} S^{2}C) 
\end{equation}

\begin{equation} \label{rawlsB}
B = -3.140 - 2.22 \times 10^{-3} \; C^{2} - 3.484 \times 10^{-5} \; S^{2} C
\end{equation}

\begin{equation} \label{rawlsPsi}
\Psi = 10.0 - \frac{(\theta - \theta_{10})(10.0 - \Psi_{e})}{\theta_{s} - \theta_{10}}
\end{equation}

\begin{equation} \label{rawlsTheta}
\theta_{10} = exp(\frac{2.302 - ln A}{B})
\end{equation}

\begin{equation} \label{rawls3}
\Psi_{e} = 100.0 (m + n \; \theta_{s})
\end{equation}

\begin{equation} \label{rawls4}
\theta_{s} = h + j \; S + k \; log_{10}(C)
\end{equation}

where $S$ is percent sand, $C$ is percent clay, __(need i include the subscripted thetas and psi?)__.

For values above 10 kPa

\begin{equation} \label{highPsi}
\Psi = A \; \theta^{B}
\end{equation}

# Gas diffusion coefficient (effective diffusivity) #

Using data from previous studies, Molderup et al. \cite{moldrup2000} first determeined a reference-potential gas diffusivity relating measured gas diffusivities to air filled porosities at a $\Psi = -100$ cm H$_2$O, $D_{P, 100}$ and $\epsilon_{100}$ respectively. ($\epsilon = 1 - \theta$).

\begin{equation} \label{eq:1}
\frac{D_{P, 100}}{D_{0}} = 2 \epsilon_{100}^{3} + 0.4 \epsilon_{100} 
\end{equation}
where $D_0$ is the gas diffusion coefficient in free air.  The above equation (eq\ref{eq:1}) was then inserted into the Burdine–Campbell relative, unsaturated hydraulic conductivity model \cite{burdine1953, cambell1974} to get,

\begin{equation} \label{eq:5}
    \frac{D_{P}}{D_{0}} = (2 \; \epsilon_{100}^{3} + 0.04 \; \epsilon_{100}) \; \left( \frac{\epsilon}{\epsilon_{100}} \right)^{2+3/b}
\end{equation}

Using equation \ref{eq:5} it is no longer necessary to have a reference point measurement, but only two parameters: $b$, the soil water retention parameter which is the slope of $\log{\epsilon}$ vs.  $\log{-\Psi}$ \cite{campbell1974}; and $\epsilon_{100}$

Normally SWC must be known for at least two values of $\Psi$ (one needing to be $\Psi_{100}$), though more are desirable for determining $b$.  In our bayesian model we will be drawing particle size fractions from a distribution and using those values to calculate the relationship between $\Psi$ based on  $\epsilon$  using the method described above.  There will thus be a distribution of values for $\Psi$, and a a resulting distribution of values for $\epsilon$. The log-log regressing to find b can be performed to find a posterior for b. 


